In [1]:
import pandas as pd
import numpy as np
import os
import math
import calendar

In [2]:
regency = os.listdir("product")
regency

['Blitar',
 'Cirebon',
 'Indramayu',
 'Jember',
 'Jombang',
 'Karawang',
 'Kediri',
 'Madiun',
 'Sukabumi',
 'Lampung Selatan',
 'Lampung Tengah',
 'Lampung Timur',
 'Majalengka',
 'Malang',
 'Metro',
 'Nganjuk',
 'Ngawi',
 'Purwakarta']

In [3]:
index_dry_season = list(range(6,18))
index_dry_season[:3]

[6, 7, 8]

In [4]:
gross_premi = 0.035

In [5]:
premium_price = [5000, 10000, 20000, 50000, 100000, 200000]

In [6]:
def planting_window(product_structure):
    planting_window_month_index = list(map(
    lambda x: int(x[3:]), product_structure['planting_window']
    ))

    month_list = list(calendar.month_name)[1:]
    month_list = list(map(
        lambda x: x[:3], month_list
    ))

    date_range = ["1-15", "16-30", "16-31", "16-29"]

    longer_month = [1,3,5,7,8,10,12]
    planting_window_month_list = []

    for i in planting_window_month_index:
        planting_window_month = month_list[math.ceil(i/2) - 1]
        
        if i%2 == 1:
            range_date = date_range[0]
        elif i%2 ==0 and (math.ceil(i/2)) == 2:
            range_date = date_range[3]
        elif i%2 == 0 and (math.ceil(i/2)) in longer_month:
            range_date = date_range[2]
        else:
            range_date = date_range[1]
        
        planting_window_month = range_date + " " + planting_window_month
        planting_window_month_list.append(planting_window_month)
    
    return(planting_window_month_list)

In [7]:
structure_dir = 'product structure dry-wet combination'
if not os.path.exists(structure_dir):
    path = os.path.join(os.getcwd(), structure_dir)
    os.mkdir(path)
for j in range(len(premium_price)):
    final_structure_table = pd.DataFrame()
    for k in regency:
        product_dir = "product\\" + k
        product_list = os.listdir(product_dir)
        for i in product_list:
            product_trigger = pd.read_csv(product_dir + "\\" + i + "\\rainy_days_triggers.csv")
            product_trigger.drop(index_dry_season, inplace=True)

            product_trigger.reset_index(inplace=True)
            product_trigger.drop(columns=['index'], inplace=True)

            table_row = len(product_trigger)

            product_trigger.insert(0, "date range", planting_window(product_trigger))
            product_trigger.insert(0, "pixel", [i]*table_row)
            product_trigger.insert(0, "regency", [k]*table_row)
            product_trigger['premium rate'] = [gross_premi]*table_row
            product_trigger['max cover'] = [0.9]*table_row
            product_trigger['premium price'] = [premium_price[j]]*table_row
            product_trigger['sum insured'] = [premium_price[j]/gross_premi]*table_row
            product_trigger["claim per day"] = [0.9/100*premium_price[j]/gross_premi]*table_row
            product_trigger['minimum rainy days'] = [4]*table_row
        
            final_structure_table = pd.concat([final_structure_table, product_trigger], ignore_index=True)
        
        final_structure_table.to_csv(
            structure_dir + "\\excess_rainfall_structure_table_" + str(premium_price[j]) + ".csv", index=False
        )